# A smaller example of prior variance scalar estimate

This is continuation of [previous notebook]() but using smaller example and more explicit code to show the problem. The data can be [downloaded here]().

The problems from previous notebook are:

1. EM converges slowly
2. The estimated prior variane scalar seems smaller than what we expect based on the coefficient estimate itself.

This notebook focuses on understanding 2. Therefore we use `optim` based result and manually check for a couple of iterations the inference for the 1st and 2nd moments, see if they are in agreement with the estimates.

In [1]:
attach(readRDS('em_optim_difference.rds'))

Here I use 15 conditions (this is enough conditions to show EM slow convergence issue),

In [2]:
R = 15
Y1 = Y[, c(1:R)]
resid_Y1 = resid_Y[c(1:R), c(1:R)]
V1 = matrix(1,R,R)
L = 1
true_coef = meta$true_coef[,c(1:R)]

To simplify, I use codes from [`BayesianMultivariateRegression` class from mmbr package](https://github.com/stephenslab/mmbr/blob/master/R/bayesian_multivariate_regression.R) which is a single component multivariate prior (not a mixture prior).

## IBSS done manually

Here I run single effect models manually for `L=2`, using `save_var` to save posterior covariance matrix

In [3]:
devtools::load_all('~/GIT/software/mmbr')
d = DenseData$new(X,Y1)
d$standardize(TRUE,TRUE)
m1 = SingleEffectModel(BayesianMultivariateRegression)$new(ncol(X), resid_Y1, V1)
m2 = SingleEffectModel(BayesianMultivariateRegression)$new(ncol(X), resid_Y1, V1)

Loading mmbr

Loading required package: mashr

Loading required package: ashr



### The first iteration

In [4]:
d$add_to_residual(m1$predict(d))
m1$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m1$predict(d))
d$add_to_residual(m2$predict(d))
m2$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m2$predict(d))

In [5]:
fitted = d$compute_Xb(m1$posterior_b1 + m2$posterior_b1)
d$compute_residual(fitted)

Now take a look at position 129, whose signal is captured by `L2` with PIP > 0.9, to check the 2nd moment estimate and prior variance scalar,

In [6]:
m2$pip[129]

[1] 0.9898636

In [7]:
sum(diag(tcrossprod(m2$posterior_b1[129,])))/R

[1] 0.001666089

In [8]:
m2$prior_variance

[1] 0.001704668

### The 2nd iteration

In [9]:
d$add_to_residual(m1$predict(d))
m1$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m1$predict(d))
d$add_to_residual(m2$predict(d))
m2$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m2$predict(d))

In [10]:
m2$pip[129]

[1] 0.9999121

In [11]:
sum(diag(tcrossprod(m2$posterior_b1[129,])))/R

[1] 0.003125922

In [12]:
m2$prior_variance

[1] 0.003131376

## The 3rd iteration

In [ ]:
d$add_to_residual(m1$predict(d))
m1$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m1$predict(d))
d$add_to_residual(m2$predict(d))
m2$fit(d, estimate_prior_variance_method = 'optim', save_var = TRUE)
d$remove_from_residual(m2$predict(d))

In [ ]:
sum(diag(tcrossprod(m2$posterior_b1[129,])))/R

In [ ]:
m2$prior_variance

It seems to be close to convergence. The estimate from this model is different from the simulated truth,

In [ ]:
m2$posterior_b1[129,]

In [ ]:
true_coef[129,]

I think the disagreement is understandable because the "all shared" prior model is very much mis-specified for these true coefficients.